***
<div style="text-align:center">
     <h1 align="center">
         <i class="fas fa-calendar-day" aria-hidden="true">  Day 14 of #3D_vision_journey </i>
    </h1>      
</div>

<h3 style='text-align: center;'>
    <img  src="../images/open3d_logo.png" height="50"/>

    ICP registration
</h3>

* Load the libraries

In [1]:
import open3d as o3d
import numpy as np
from   copy import copy, deepcopy

draw_bit = 0 # draw it or not!

###  Load point cloud

In [2]:
print("Load a point cloud, and render it...\n")
source = o3d.io.read_point_cloud("../data/pointcloud/cloud_bin_0.pcd")
target = o3d.io.read_point_cloud("../data/pointcloud/cloud_bin_1.pcd")

print(f"""The 'source' have {np.asarray(source.points).shape[0]} 3D points,
The 'target' have {np.asarray(target.points).shape[0]} 3D points""")

if draw_bit:
    o3d.visualization.draw_geometries([source])


Load a point cloud, and render it...

The 'source' have 198835 3D points,
The 'target' have 137833 3D points


### transformation matrix

the initial transformation is a 
- 4x4 matrix(`[R, t; 0, 1]`), 
- `R` is a 3x3 `rotation matrix` and 
- `t` is a 3x1 `translation vector`. 

In [3]:
threshold  = 0.02
# Manually set the transformation matrix
trans_init = np.asarray([[0.862,  0.011, -0.507,  0.5],
                         [-0.139, 0.967, -0.215,  0.7],
                         [0.487,  0.255,  0.835, -1.4], 
                         [0.0,    0.0,     0.0,   1.0]])
print(f"The initial transformation matrix:\n{trans_init}")

The initial transformation matrix:
[[ 0.862  0.011 -0.507  0.5  ]
 [-0.139  0.967 -0.215  0.7  ]
 [ 0.487  0.255  0.835 -1.4  ]
 [ 0.     0.     0.     1.   ]]


### Draw registration result



In [4]:
def draw_registration_result(source, target, transformation):
    source_temp = deepcopy(source)
    target_temp = deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    # visualizing
    o3d.visualization.draw_geometries([source_temp, target_temp],
                                        zoom=0.4459,
                                        front=[0.9288, -0.2951, -0.2242],
                                        lookat=[1.6784, 2.0612, 1.4451],
                                        up=[-0.3402, -0.9189, -0.1996])
if draw_bit:
    
    draw_registration_result(source, target, trans_init)

### Initial alignment
- `fitness`:    The higher,   the better
- `inlier_rms`: The lower,    the better

In [5]:
#%%timeit

evaluation = o3d.pipelines.registration.evaluate_registration(source, target, threshold, trans_init)

if draw_bit:
    print(evaluation,'\n') 


### Apply point-to-point ICP

In [6]:
#%%timeit

print("Apply point-to-point ICP")
reg_p2p = o3d.pipelines.registration.registration_icp(
    source, target, threshold, trans_init,
    o3d.pipelines.registration.TransformationEstimationPointToPoint())

if draw_bit:
    print(reg_p2p)
    print("\nTransformation is:")
    print(reg_p2p.transformation)
    draw_registration_result(source, target, reg_p2p.transformation)

Apply point-to-point ICP


### P-t-P ICP with 2000 iterations

In [7]:
#%%timeit

print("P-t-P ICP with 2000 iterations")
reg_p2p = o3d.pipelines.registration.registration_icp(
    source, target, threshold, trans_init,
    o3d.pipelines.registration.TransformationEstimationPointToPoint(),
    o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=2000))

if draw_bit:
    print(reg_p2p)
    draw_registration_result(source, target, reg_p2p.transformation)

P-t-P ICP with 2000 iterations


### Point-to -Plane ICP

In [8]:

#%%timeit

print("Apply point-to-plane ICP")
reg_p2l = o3d.pipelines.registration.registration_icp(
    source, target, threshold, trans_init,
    o3d.pipelines.registration.TransformationEstimationPointToPlane())
    
if draw_bit:
    print(reg_p2l)
    print("Transformation is:")
    print(reg_p2l.transformation)
    draw_registration_result(source, target, reg_p2l.transformation)

Apply point-to-plane ICP


***
<div style="text-align:center">
     <h1 align="center">
         <i class="fas fa-calendar-day" aria-hidden="true">  Day 15 of #3D_vision_journey </i>
    </h1>      
</div>

<h3 style='text-align: center;'>
    <img  src="../images/open3d_logo.png" height="50"/> 
    
    Robust ICP   
</h3>

### Source PointCloud + noise

In [9]:
def apply_noise(pcd, mu, sigma):
    noisy_pcd = deepcopy(pcd)
    points = np.asarray(noisy_pcd.points)
    points += np.random.normal(mu, sigma, size=points.shape)
    noisy_pcd.points = o3d.utility.Vector3dVector(points)
    return noisy_pcd

mu, sigma = 0, 0.1  # mean and standard deviation
source_noisy = apply_noise(source, mu, sigma)

if draw_bit:
    print("Source PointCloud + noise:")
    o3d.visualization.draw_geometries([source_noisy],
                                    zoom=0.4459,
                                    front=[0.353, -0.469, -0.809],
                                    lookat=[2.343, 2.217, 1.809],
                                    up=[-0.097, -0.879, 0.467])

### Vanilla point-to-plane ICP

In [10]:
#%%timeit

threshold = 1.0
print("Vanilla point-to-plane ICP, threshold={}:".format(threshold))
p2l = o3d.pipelines.registration.TransformationEstimationPointToPlane()
reg_p2l = o3d.pipelines.registration.registration_icp(source_noisy, target,
                                                      threshold, trans_init,
                                                      p2l)
if draw_bit:
    print(reg_p2l)
    print("\nTransformation is:")
    print(reg_p2l.transformation)
    draw_registration_result(source, target, reg_p2l.transformation)

Vanilla point-to-plane ICP, threshold=1.0:


### Robust point-to-plane ICP

In [12]:
#%%timeit

print("Robust point-to-plane ICP, threshold={}:".format(threshold))
loss = o3d.pipelines.registration.TukeyLoss(k=sigma)
print("Using robust loss:", loss)
p2l = o3d.pipelines.registration.TransformationEstimationPointToPlane(loss)
reg_p2l = o3d.pipelines.registration.registration_icp(source_noisy, target,
                                                      threshold, trans_init,
                                                      p2l)
if draw_bit:
    print(reg_p2l)
    print("\nTransformation is:")
    print(reg_p2l.transformation)
    draw_registration_result(source, target, reg_p2l.transformation)

Robust point-to-plane ICP, threshold=1.0:
Using robust loss: RobustKernel::TukeyLoss with k=0.100000
